In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./modules")

import os
import argparse
import numpy as np
import matplotlib.pyplot as plt

from Potential import Potential
from Channel import Channel
from Scattering import Scattering
from ScatteringExp import ScatteringExp
from Grid import Grid
import RseSolver
from scipy.special import factorial2, spherical_jn
from plots.rc_params import *

In [2]:
potentialArgs = {"label": "minnesota", "kwargs": {"potId": 213}}
l = 2
channel = Channel(S=0, L=l, LL=l, J=l, channel=0)
potential = Potential(channel, **potentialArgs)
E_MeV = 50
scattExp = ScatteringExp(E_MeV=E_MeV, potential=potential)
rmatch = 12
grid = Grid(1e-24, rmatch, numIntervals=1, numPointsPerInterval=1000,
            type="linear", test=False)  # needs equidistant grids as of now
trainingLecList, testingLecList = Potential.getSampleLecs(potentialArgs["label"])

emulator_args = dict(scattExp=scattExp, 
                          grid=grid, 
                          free_lecs={"V1": (-400, 0)},
                          num_snapshots_init=80, 
                          num_snapshots_max=100,
                          approach="pod", 
                          pod_rcond=1e-12, 
                          init_snapshot_lecs=None,
                          greedy_max_iter=5, 
                          inhomogeneous=True,
                          mode="linear")

In [3]:
lecs = np.array([[1,200,-91.85], [1,200,-91.85]])

In [4]:
from NumerovEmulator import MatrixNumerovROM_ab, MatrixNumerovROM, AllAtOnceNumerovROM
# Test config for no greedy
emulator_T = AllAtOnceNumerovROM(**emulator_args)
emulator_ab = MatrixNumerovROM_ab(**emulator_args)
emulator = MatrixNumerovROM(**emulator_args)

using 8 out of 80 POD modes in total: compression rate is 90.0 %
POD[ Y ]: compression rate is 37.0 %; dim: (1000, 17)
using 7 out of 80 POD modes in total: compression rate is 91.2 %
POD[ Y ]: compression rate is 33.3 %; dim: (1000, 16)
testing
using 7 out of 80 POD modes in total: compression rate is 91.2 %
POD[ Y ]: compression rate is 33.3 %; dim: (998, 16)


In [5]:
# %timeit emulator_T.simulate(lecs) 
# %timeit emulator_T.emulate(lecs, mode="lspg", lspg_solver="qr") 

In [6]:
def get_Kmat(S):
    return (1-S)/(1+S)*1j
print(get_Kmat(emulator_T.simulate(lecs, which="S")))
print(emulator_T.simulate(lecs, which="S"))
print(emulator_T.simulate(lecs, which="K"))
print(emulator_T.simulate(lecs, which="T"))
print(emulator_ab.simulate(lecs, which="K"))
print(emulator.simulate(lecs, which="S"))

[0.12807611+6.38821773e-11j 0.12807611+6.38821773e-11j]
[0.96772248+0.25201824j 0.96772248+0.25201824j]
[0.12807611+6.3882153e-11j 0.12807611+6.3882153e-11j]
[0.12600912+0.01613876j 0.12600912+0.01613876j]
[0.12807611 0.12807611]
[0.96772248+0.25201824j 0.96772248+0.25201824j]


In [7]:
def get_Kmat(S):
    return (1-S)/(1+S)*1j
# print(get_Kmat(emulator_T.emulate(lecs, which="S")))
# print(emulator_T.emulate(lecs, which="S"))
# print(emulator_T.emulate(lecs, which="K"))
# print(emulator_T.emulate(lecs, which="T"))
# print(emulator_ab.emulate(lecs, which="K"))
print(emulator.emulate(lecs, which="K"))

[[-0.24216361 -0.24216361]
 [ 0.10966459  0.10966459]]
diff1 [[ 2.77555756e-17  2.77555756e-17]
 [-2.77555756e-17 -2.77555756e-17]]
diff2 [[2.77555756e-17 2.77555756e-17]
 [0.00000000e+00 0.00000000e+00]]
diff3 [[0.00000000e+00 0.00000000e+00]
 [2.77555756e-17 2.77555756e-17]]

[[0.02580365 0.02580365]
 [0.02894738 0.02894738]
 [0.03208625 0.03208625]
 [0.03521974 0.03521974]
 [0.03834731 0.03834731]
 [0.04146845 0.04146845]
 [0.04458263 0.04458263]
 [0.04768933 0.04768933]
 [0.05078802 0.05078802]
 [0.05387819 0.05387819]
 [0.05695931 0.05695931]
 [0.06003087 0.06003087]
 [0.06309235 0.06309235]
 [0.06614323 0.06614323]
 [0.06918301 0.06918301]
 [0.07221116 0.07221116]
 [0.07522719 0.07522719]
 [0.07823058 0.07823058]
 [0.08122083 0.08122083]
 [0.08419744 0.08419744]
 [0.0871599  0.0871599 ]
 [0.09010771 0.09010771]
 [0.09304039 0.09304039]
 [0.09595743 0.09595743]
 [0.09885834 0.09885834]]
[0.12807611 0.12807611]


In [8]:
emulator_ab.simulate(lecs, which="all")[-2:]

array([[-0.2204676 , -0.2204676 ],
       [ 0.09983948,  0.09983948]])

In [9]:
# print(emulator_T.emulate(lecs, which="all")[-1])
print(emulator_ab.emulate(lecs, which="K"))
# print(emulator.emulate(lecs, which="T"))

[0.12807611 0.12807611]


In [10]:
np.max(np.abs(emulator.simulate(lecs) - emulator_ab.simulate(lecs)[:-2,:]))

5.9396931817445875e-15

In [11]:
aa = emulator_T.emulate_Tmatrix(lecs, estimate_norm_residual=True)
1 + 2j * aa

AttributeError: 'AllAtOnceNumerovROM' object has no attribute 'emulate_Tmatrix'

In [ ]:
Smat = emulator_T.emulate_Smatrix(lecs, estimate_norm_residual=True)
# np.abs(emulator_T.emulate_Smatrix(lecs, estimate_norm_residual=True))
np.log(Smat)/(2j)


array([-0.50738448+0.43454838j])

In [ ]:
np.max(np.abs(emulator_T.simulate(lecs)[:-1,:] - emulator_ab.simulate(lecs)[:-2,:]))

1.556367162194613

In [ ]:
aaa = np.array([1,2,3])
a = aaa[0]
a = 333
aaa

array([1, 2, 3])